# Подбор гипер параметров

В модели встроена поддержка optuna для подбора гипер параметров моделей.

In [1]:
import pandas as pd
df = pd.read_csv("data/ml_ratings.csv")
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
from replay.data_preparator import DataPreparator

log = DataPreparator().transform(
    data=df,
    columns_names={
        "user_id": "userId",
        "item_id": "movieId",
        "timestamp": "timestamp",
        "relevance": "rating"
    }
).cache()

In [5]:
from replay.splitters import UserSplitter

splitter = UserSplitter(
    item_test_size=1,
    user_test_size=0.2,
    drop_cold_items=True,
    drop_cold_users=True,
    shuffle=False,
    seed=1234
)

train, test = splitter.split(log)

# Пример на KNN

Если необходимо затюнить конкретную модель, необходимо создать инстанс модели и воспользоваться методом `optimize`.

Из обязательных параметров — `train` и `test`.

Границы перебора задаются с помощью словаря `param_grid` вида `{param: [low, high]}`. 

Можно не передавать сетку, тогда будет использоваться предопределенная для этой модели в атрибуте `_search_space`. 
Посмотрим на него, чтобы понять, что можно перебирать для данной модели и какие границы нам показались разумными.

In [6]:
from replay.models import KNN
model = KNN()
model._search_space

/Users/darel/python/sponge-bob-magic/.new_env2/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


{'num_neighbours': {'type': 'int', 'args': [1, 100]},
 'shrink': {'type': 'int', 'args': [0, 100]}}

In [7]:
model.optimize(train, test, budget=2, param_grid={'num_neighbours': [10, 20]})

[I 2021-03-02 17:39:19,952] A new study created in memory with name: no-name-74e0d1b4-7748-427e-bff8-b3e25816071d
[W 2021-03-02 17:39:19,956] Trial 0 failed because of the following error: AttributeError("'KNN' object has no attribute 'similarity'")
Traceback (most recent call last):
  File "/Users/darel/python/sponge-bob-magic/.new_env2/lib/python3.7/site-packages/optuna/_optimize.py", line 198, in _run_trial
    value_or_values = func(trial)
  File "/Users/darel/python/sponge-bob-magic/replay/optuna_objective.py", line 47, in __call__
    return self.objective_calculator(trial=trial, **self.kwargs)
  File "/Users/darel/python/sponge-bob-magic/replay/optuna_objective.py", line 135, in scenario_objective_calculator
    recommender.set_params(**params_for_trial)
  File "/Users/darel/python/sponge-bob-magic/replay/models/base_rec.py", line 110, in set_params
    self._clear_cache()
  File "/Users/darel/python/sponge-bob-magic/replay/models/knn.py", line 141, in _clear_cache
    if self.s

AttributeError: 'KNN' object has no attribute 'similarity'